In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# Conversando com os dados

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
caminhos = [
    'arquivos/Explorando o Universo das IAs com Hugging Face.pdf',
    'arquivos/Explorando a API da OpenAI.pdf'
]


paginas = []
for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=10,
    separators=['\n\n', '\n', '.', ' ', '']
)

documents = recur_split.split_documents(paginas)

for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('arquivos/', '')
    doc.metadata['doc_id'] = i

In [4]:
diretorio = 'arquivos/chat_retrieval_bd'

embeddigs_model = OpenAIEmbeddings()
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddigs_model,
    persist_directory=diretorio
)

## Criando estrutura de conversa

In [5]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI(model='gpt-4o-mini')

In [6]:
from langchain.chains.retrieval_qa.base import RetrievalQA

chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr')
)

In [7]:
pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
chat_chain.invoke({'query': pergunta})

{'query': 'O que é Hugging Face e como faço para acessá-lo?',
 'result': 'Hugging Face é uma plataforma que oferece uma variedade de modelos de inteligência artificial, ferramentas e recursos voltados para o desenvolvimento e aplicação de IAs, especialmente em tarefas de processamento de linguagem natural. A plataforma é conhecida por sua comunidade ativa e por fornecer acesso a modelos e datasets.\n\nPara acessar o Hugging Face, você pode seguir os seguintes passos:\n\n1. Visite o site oficial do Hugging Face.\n2. Crie uma conta, se ainda não tiver uma.\n3. Após fazer login, você poderá explorar diversos modelos, datasets e recursos disponíveis na plataforma.\n4. Se necessário, você pode solicitar acesso a modelos restritos, seguindo as instruções específicas na página do modelo.\n\nAlém disso, você pode participar da comunidade, seguir perfis, acessar artigos e cursos, e interagir no servidor de Discord, fórum e GitHub oficial.'}

## Modificando o prompt da chain

In [8]:
from langchain.prompts import PromptTemplate

chain_prompt = PromptTemplate.from_template(
'''Utilize o contexto fornecido para responder a pergunta final.
Se você não sabe a resposta, apenas diga que não sabe e não tente inventar a resposta.
Utilize três frases no máximo, mantenha a resposta concisa.

Contexto: {context}

Pergunta: {question}
'''
)

In [9]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type_kwargs={'prompt': chain_prompt},
    return_source_documents=True
)

In [11]:
pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

Hugging Face é uma plataforma que oferece uma variedade de modelos de inteligência artificial e ferramentas para desenvolvedores. Para acessá-lo, você deve criar uma conta e, se necessário, solicitar acesso a modelos restritos, concordando com os termos de uso. Após isso, você pode explorar e utilizar os modelos disponíveis na plataforma.


In [12]:
print(resposta['source_documents'])

[Document(metadata={'doc_id': 1, 'page': 1, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}, page_content='Explorando o Universo das IAs com Hugging Face\nConteúdo\n01. O que é Hugging Face? 5\nAfinal, o que é Hugging Face? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\nÉ tudo aberto mesmo? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6\nComo usaremos o Hugging Face? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6\n02. A plataforma Hugging Face 7\nModelos . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7\nPágina de um modelo . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 8\nDatasets . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 10\nSpaces . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 10\n03. Testando nossa primeira IA em tempo recorde 14\nNossa p

In [13]:
from langchain.globals import set_debug

set_debug(True)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})

set_debug(False)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "O que é Hugging Face e como faço para acessá-lo?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "O que é Hugging Face e como faço para acessá-lo?",
  "context": "Explorando o Universo das IAs com Hugging Face\nConteúdo\n01. O que é Hugging Face? 5\nAfinal, o que é Hugging Face? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\nÉ tudo aberto mesmo? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6\nComo usaremos o Hugging Face? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6\n02. A plataforma Hugging Face 7\nModelos . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7\nPágina de um modelo . . . . . . . . . . . . . . . . . . . . 

## Outros tipos de chains

### Stuff

In [14]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever = vectordb.as_retriever(search_type='mmr'),
    chain_type='stuff'
)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

Hugging Face é uma plataforma que oferece uma variedade de modelos de inteligência artificial, especialmente para tarefas de processamento de linguagem natural. É uma comunidade colaborativa onde os usuários podem compartilhar modelos, datasets e ideias.

Para acessar o Hugging Face, você pode criar uma conta no site oficial. Após criar sua conta e fazer login, você poderá explorar diferentes modelos, datasets e opções da comunidade. Para alguns modelos que são restritos, você pode precisar solicitar acesso, o que geralmente envolve ler e concordar com os termos de uso.

Se precisar de mais detalhes sobre como utilizar a plataforma, você pode consultar a documentação ou os tutoriais disponíveis no site.


### Map Reduce

In [15]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever = vectordb.as_retriever(search_type='mmr'),
    chain_type='map_reduce'
)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

Hugging Face é uma plataforma que fornece ferramentas e modelos para o desenvolvimento de aplicações de inteligência artificial, especialmente em processamento de linguagem natural. Ele oferece uma variedade de modelos que permitem aos usuários realizar tarefas específicas relacionadas à inteligência artificial.

Para acessar o Hugging Face, você deve abrir a página inicial da plataforma enquanto está logado. Nela, você encontrará diversas opções relacionadas à comunidade e aos modelos disponíveis. Para acessar um modelo específico, você deve voltar à página do modelo desejado e clicar em um botão para solicitar acesso, dependendo do modelo. O processo pode variar, e em alguns casos, como o modelo do Google, pode ser necessário ler e concordar com os termos de uso. 

Além disso, você pode buscar por tarefas específicas e seguir o processo de avaliação para rodar os modelos localmente. É possível escolher modelos que atendam às suas necessidades e integrá-los com outros scripts ou progr

### Refine

In [16]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever = vectordb.as_retriever(search_type='mmr'),
    chain_type='refine'
)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

Hugging Face é uma plataforma que oferece uma ampla gama de ferramentas e modelos para trabalhar com inteligência artificial (IA), especialmente no campo do processamento de linguagem natural (NLP). A plataforma é famosa por sua biblioteca de modelos pré-treinados, que podem ser facilmente utilizados para diversas aplicações de IA, permitindo que desenvolvedores e pesquisadores explorem novas tarefas e integrem diferentes modelos em suas aplicações.

Para acessar o Hugging Face, você pode visitar o site oficial (huggingface.co) e criar uma conta, caso ainda não tenha uma. Após a criação da conta, você terá a oportunidade de explorar os modelos disponíveis e as tarefas que o Hugging Face oferece. O menu à direita da página inicial permite acesso a diversos recursos comunitários, incluindo artigos de blog, cursos oferecidos pelo Hugging Face, e links para o servidor do Discord, o Fórum e o GitHub oficial.

Uma vez na plataforma, você pode procurar por novas tarefas, encontrar o código ne